In [1]:
%pip install -Uq "unstructured[all-docs]"
%pip install -Uq langchain_chroma
%pip install -Uq langchain langchain-community langchain-openai
%pip install -Uq python_dotenv
%pip install hf_xet

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import json
from typing import List

from unstructured.partition.pdf import partition_pdf
from unstructured.chunking.title import chunk_by_title

from langchain_core.documents import Document
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_core.messages import HumanMessage
from dotenv import load_dotenv

load_dotenv()

c:\Users\Reyad\Documents\Mulimodal_RAG_Langchain\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [3]:
def partition_document(file_path:str):
    print(f"partition document: {file_path}")

    elements = partition_pdf(
        filename = file_path,
        strategy="hi_res",
        infer_table_structure=True,
        extract_image_block_types=["Image"],
        extract_image_block_to_payload=True
    )

    print(f"Extracted {len(elements)} elements")
    return elements

In [4]:
file_path="3723178.3723198.pdf"
elements = partition_document(file_path=file_path)

partition document: 3723178.3723198.pdf


The `max_size` parameter is deprecated and will be removed in v4.26. Please specify in `size['longest_edge'] instead`.


Extracted 195 elements


In [5]:
elements

In [6]:
len(elements)

195

In [9]:
set([str(type(el)) for el in elements])

{"<class 'unstructured.documents.elements.FigureCaption'>",
 "<class 'unstructured.documents.elements.Footer'>",
 "<class 'unstructured.documents.elements.Header'>",
 "<class 'unstructured.documents.elements.Image'>",
 "<class 'unstructured.documents.elements.ListItem'>",
 "<class 'unstructured.documents.elements.NarrativeText'>",
 "<class 'unstructured.documents.elements.Table'>",
 "<class 'unstructured.documents.elements.Text'>",
 "<class 'unstructured.documents.elements.Title'>"}

In [11]:
elements[2].to_dict()

{'type': 'NarrativeText',
 'element_id': '97c93816158eddc5f5f562c74c203a6c',
 'text': 'Al-Momen Reyad Computer Science and Engineering United International University Dhaka, Bangladesh areyad203011@bscse.uiu.ac.bd',
 'metadata': {'detection_class_prob': 0.8866927027702332,
  'coordinates': {'points': ((np.float64(195.79999999999998),
     np.float64(353.9244444444443)),
    (np.float64(195.79999999999998), np.float64(525.2470703125)),
    (np.float64(595.0235211111112), np.float64(525.2470703125)),
    (np.float64(595.0235211111112), np.float64(353.9244444444443))),
   'system': 'PixelSpace',
   'layout_width': 1700,
   'layout_height': 2200},
  'links': [{'text': 'Al - Momen Reyad',
    'url': 'https://orcid.org/0009-0005-3905-6073',
    'start_index': 0},
   {'text': 'Al - Momen Reyad',
    'url': 'https://orcid.org/0009-0005-3905-6073',
    'start_index': 0}],
  'last_modified': '2025-09-14T12:23:44',
  'filetype': 'application/pdf',
  'languages': ['eng'],
  'page_number': 1,
  'fi

In [12]:
images = [element for element in elements if element.category=="Image"]

In [13]:
len(images)

12

In [14]:
images[1].to_dict()

{'type': 'Image',
 'element_id': '35c4a2c37e25128d7785dae3c46841a7',
 'text': 'Train(80%) Coccidiosisd Salmonellosis Healthy New- castle disease Data Augmentation + Rotation ightmess adjustment + Slight image distortion Dataset Test(20%) Classes Data preprocessing Prediction Majority voting Model Train',
 'metadata': {'coordinates': {'points': ((np.float64(149.4388888888889),
     np.float64(978.5530902777778)),
    (np.float64(149.4388888888889), np.float64(1452.5777777777778)),
    (np.float64(1550.5450347222222), np.float64(1452.5777777777778)),
    (np.float64(1550.5450347222222), np.float64(978.5530902777778))),
   'system': 'PixelSpace',
   'layout_width': 1700,
   'layout_height': 2200},
  'last_modified': '2025-09-14T12:23:44',
  'filetype': 'application/pdf',
  'languages': ['eng'],
  'page_number': 3,
  'image_base64': '...',
  'image_mime_type': 'image/jpeg',
  'filename': '3723178.3723198.pdf'}}

In [15]:
tables = [element for element in elements if element.category=="Table"]
len(tables)

4

In [16]:
tables[1].to_dict()

{'type': 'Table',
 'element_id': '6cdff955deae329e98d80f3bab635b8f',
 'text': 'Model Classes Precision Recall F1-score Accuracy Ncd 0.98 0.93 0.95 0.96 VGG19 Salmo 0.96 0.94 0.95 0.96 Healthy 0.92 0.98 0.95 0.96 Cocci 0.99 0.98 0.98 0.96 Ncd 0.98 0.99 0.98 0.91 MobileNetV3 Salmo 0.76 1.00 0.87 0.91 Healthy 1.00 0.67 0.80 0.91 Cocci 1.00 0.98 0.99 0.91 Ncd 1.00 1.00 1.00 0.99 Swin Transformer Salmo 0.97 0.99 0.98 0.99 Healthy 0.99 0.97 0.98 0.99 Cocci 1.00 1.00 1.00 0.99 Ncd 0.98 0.99 0.99 0.99 Salmo 0.98 0.99 0.99 0.99 Healthy 0.98 0.98 0.98 0.99 Cocci 1.00 0.99 0.99 0.99 Ncd 0.93 0.96 0.95 0.95 EfficientNet B7 Salmo 0.96 0.94 0.95 0.95 Healthy 0.94 0.92 0.93 0.95 Cocci 0.97 0.98 0.98 0.95',
 'metadata': {'detection_class_prob': 0.915313720703125,
  'coordinates': {'points': ((np.float64(458.08538818359375),
     np.float64(302.75775146484375)),
    (np.float64(458.08538818359375), np.float64(1197.3492431640625)),
    (np.float64(1259.544921875), np.float64(1197.3492431640625)),
    (n

In [17]:
def create_chunks_by_title(elements):
    chunks = chunk_by_title(
        elements=elements,
        # character not more than 3000, this one is hard limit
        max_characters=3000,
        # try to make new chunk affter 2400 chracter, this is one is soft limit
        new_after_n_chars=2400,
        # if charater is less then 500 and merge
        combine_text_under_n_chars=500 
    )
    
    print(f"Created total {len(chunks)} chunks")
    return chunks

chunks = create_chunks_by_title(elements)

Created total 21 chunks


In [18]:
chunks

In [19]:
set([str(type(chunk)) for chunk in chunks])

{"<class 'unstructured.documents.elements.CompositeElement'>"}

In [28]:
chunks[11].to_dict()

{'type': 'CompositeElement',
 'element_id': '82df9bd6-fda1-4b5d-aa12-84ad62b16926',
 'text': 'EfficientNetB7 architecture is pre-trained on ImageNet, exclud-\n\ning the top classification layer. The model consists 4 different types of layer. Like firstly an additional layers including batch normaliza- tion [20], a dense layer with ReLU activation (256 units), a dropout layer (dropout rate: 0.2) and a final dense layer with softmax activa- tion for classification. Training is facilitated using the Adamax opti- mizer (learning rate: 0.001) and categorical cross-entropy loss, with accuracy serving as the evaluation metric. The model parameters\n\nThese individual models include a range of architecture and\n\neach is learned separately. Each of them takes an input image and provides its own prediction on what class label the image belongs to. After that, majority voting strategy is imposed for compiling together these predictions, where out of all the given predictions the most common one 

In [29]:
chunks[11].metadata.orig_elements

In [30]:
chunks[11].metadata.orig_elements[9].to_dict()

{'type': 'Image',
 'element_id': '1e8d057ecdce2aafc421f83a1902ab85',
 'text': 'Confusion Matrix i 400 8 8 2 300 2 o oe o 2 8 o 2 200 fs 3 ij & -100 ° E rf g -0 cocci healthy ned salmo Predicted labels',
 'metadata': {'coordinates': {'points': ((np.float64(149.4388888888889),
     np.float64(485.7825066666666)),
    (np.float64(149.4388888888889), np.float64(1122.9166666666665)),
    (np.float64(849.9981688888889), np.float64(1122.9166666666665)),
    (np.float64(849.9981688888889), np.float64(485.7825066666666))),
   'system': 'PixelSpace',
   'layout_width': 1700,
   'layout_height': 2200},
  'last_modified': '2025-09-14T12:23:44',
  'filetype': 'application/pdf',
  'languages': ['eng'],
  'page_number': 6,
  'image_base64': '...',
  'image_mime_type': 'image/jpeg',
  'filename': '3723178.3723198.pdf'}}